In [64]:
import numpy as np
import pandas as pd
import pytz

import requests
import time
import pickle
import demoji

from datetime import datetime
from string import punctuation

In [3]:
demoji.download_codes()

... OK (Got response in 1.37 seconds)
Writing emoji data to /home/batyrkhan/.demoji/codes.json ...
... OK


In [109]:
r = requests.get(
    'https://www.reddit.com/r/wallstreetbets/.json',
    headers={'user-agent': 'Mozilla/5.0'})
#Finds current name of the pinned discussion

fresh = 'https://www.reddit.com' + r.json()['data']['children'][0]['data']['permalink'] + '.json'

In [108]:
r.json()['data']['children'][0]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'wallstreetbets',
  'selftext': 'Your daily trading discussion thread. Please keep the shitposting to a minimum. \n\n^Navigate ^WSB|^We ^recommend ^best ^daily ^DD\n:--|:--                                 \n**DD** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADD) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=week)\n**Discussion** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADiscussion) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADiscussion&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADiscussion&amp;restrict_sr=on&amp;t=week)\n**

In [106]:
print(fresh)

https://www.reddit.com/r/wallstreetbets/comments/m796cv/what_are_your_moves_tomorrow_march_18_2021/.json


In [110]:
fresh

'https://www.reddit.com/r/wallstreetbets/comments/m7nj0c/daily_discussion_thread_for_march_18_2021/.json'

In [111]:
def get_posts():
    r = requests.get(fresh, headers={'user-agent': 'Mozilla/5.0'})
    posts = r.json()[1]['data']['children']
    posts_new = []
    for post in posts:
        if 'body' not in post['data']:
            continue
        dt = datetime.fromtimestamp(post['data']['created_utc'])
        dt = datetime(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, 0, pytz.timezone('Asia/Almaty'))
        posts_new.append({'dt': dt, 'ticker_like': find_ticker_like(post['data']['body']), 'text': post['data']['body']})
#     posts = [{'dt': dt, 'ticker_like': find_ticker_like(post['data']['body']), 'text': post['data']['body']} 
#              for post in posts if 'body' in post['data']]
    return posts_new

In [73]:
def is_ticker_like(text):
    """
    Checks if the text looks like a ticker. Not 100% reliable
    """
    if (text.upper() == text) or (text[0] == '$'):
        
        text = ''.join([t for t in list(text) if t not in punctuation])
        
        if text in ['I', '', 'WSB']:
            return False
        else:
            try: 
                float(text)
                return False
            except:
                try: 
                    float(text.split('$')[1])
                    return False
                except:
                    return True
    else:
        return False
    
    pass

In [74]:
is_ticker_like('ASD')

True

In [88]:
def find_ticker_like(text):
    for candidate in text.split(' '):
        if is_ticker_like(candidate):
            return candidate
    return None

In [59]:
len(string.punctuation)

32

In [114]:
posts = get_posts()
print(len(posts))
posts

125


[{'dt': datetime.datetime(2021, 3, 18, 19, 20, 44, tzinfo=<DstTzInfo 'Asia/Almaty' LMT+5:08:00 STD>),
  'ticker_like': '$3/gallon',
  'text': "Make sure you guys keep enough cash for gas money. It's about to hit $3/gallon"},
 {'dt': datetime.datetime(2021, 3, 18, 19, 20, 43, tzinfo=<DstTzInfo 'Asia/Almaty' LMT+5:08:00 STD>),
  'ticker_like': None,
  'text': "Discovery is 130 percent shorted we pushed it from 30 to 75 let's get these hedge fund bastards we have 4 days and then they have to buy up their shorts"},
 {'dt': datetime.datetime(2021, 3, 18, 19, 20, 41, tzinfo=<DstTzInfo 'Asia/Almaty' LMT+5:08:00 STD>),
  'ticker_like': None,
  'text': 'Still not buying bonds'},
 {'dt': datetime.datetime(2021, 3, 18, 19, 20, 34, tzinfo=<DstTzInfo 'Asia/Almaty' LMT+5:08:00 STD>),
  'ticker_like': None,
  'text': 'Bears can’t short what you don’t sell. Buy calls.'},
 {'dt': datetime.datetime(2021, 3, 18, 19, 20, 26, tzinfo=<DstTzInfo 'Asia/Almaty' LMT+5:08:00 STD>),
  'ticker_like': 'TLT',
  'tex